In [1]:
import gensim
from gensim.models import KeyedVectors
import collections
from collections import OrderedDict
import os



In [2]:
path = '/home/ayan-yue/Documents/projects/dynamic-word-embeddings/diachronic-models/'

In [3]:
dyn_vec = OrderedDict()
t = 19

for i in range(t):
    for file in os.listdir(path):
        filename = os.path.join(path, file)
        
        if os.path.basename(filename).split('.')[0][1:] == str(i):
            dyn_vec['t{0}'.format(i)] = KeyedVectors.load(filename)
    
    

In [4]:
import pickle

with open('pickled_d', 'rb') as fp:
    d = pickle.load(fp)


In [5]:
for key in d:
    d[key] = d[key].split(' ')
    

In [21]:
def vocab_search(string):
    
    slices = []
    for key in d:
        for i in range(len(d[key])):
            if d[key][i] == string:
                slices.append(key)
                break
    
    return slices
            

In [22]:
vocab_search('government')

['t0',
 't2',
 't4',
 't5',
 't6',
 't7',
 't8',
 't9',
 't10',
 't11',
 't12',
 't13',
 't15',
 't16',
 't17',
 't18']

In [25]:
dyn_vec['t18'].most_similar('government')

/home/ayan-yue/Documents/projects/env1/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('figure,', 0.989435076713562),
 ('stages', 0.9841992855072021),
 ('approximately', 0.98386549949646),
 ('office.', 0.9837332963943481),
 ('nhmf', 0.983287513256073),
 ('home', 0.982962965965271),
 ('successive', 0.982195258140564),
 ('artist’s', 0.9821171164512634),
 ('extensive', 0.9819861650466919),
 ('who', 0.9817845225334167)]